In [1]:
import pandas as pd
import boto3
from io import StringIO, BytesIO #In memory buffer to use the read_csv method
from datetime import datetime as dt,timedelta,date


In [2]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [3]:
#  Application Layer


def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format,meta_key,extract_date_list):
    key = trg_key + dt.today().strftime("%Y-%m-%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket,meta_key,extract_date_list)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format,meta_key):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date>=arg_date]
    load(trg_bucket, df, trg_key, trg_format,meta_key,extract_date_list)
    return True
    

In [4]:
# Secondary Application Layer
def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = dt.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = dt.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').execptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket,meta_key, extract_date_list):
    df_meta_new = pd.DataFrame(columns= ['source_date','datetime_of_processing'])
    df_meta_new['source_date']= extract_date_list
    df_meta_new['datetime_of_processing']=dt.today().strftime('%Y-%m-%d')
    df_meta_old = read_csv_to_df(bucket, meta_key)
    df_meta_new_all=pd.concat([df_meta_old,df_meta_new])
    write_df_to_s3_csv(bucket,df_meta_new_all,meta_key)
    

In [5]:
# Main Function Entry Point

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-01-03'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-abhijit'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format, meta_key)

In [6]:
# Run

main()

## Read from Parquet

In [7]:
#Using Boto for python to access the s3 resource
s3=boto3.resource('s3')
#Initializing source bucket for the AWS repository name

bucket_trg=s3.Bucket('xetra-abhijit')

df_stock_report_complete=pd.DataFrame()
for obj in bucket_trg.objects.all():
    
    print(obj.key)

    

meta_file.csv
xetra_daily_report_2022-01-05_185637.parquet
xetra_daily_report_2022-01-06_124256.parquet
xetra_daily_report_2022-01-06_133621.parquet
xetra_daily_report_2022-01-08_161303.parquet
xetra_daily_report_2022-01-08_161517.parquet
xetra_daily_report_2022-01-08_162513.parquet
xetra_daily_report_20220108_154941.parquet


In [9]:
parquet_obj = bucket_trg.Object(key='xetra_daily_report_2022-01-08_162513.parquet').get().get('Body').read()
stock_prices=BytesIO(parquet_obj)
df_stock_report=pd.read_parquet(stock_prices)
df_stock_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
15944,XS2376095068,2022-01-03,41.50,41.04,41.00,41.83,2,NaN
15945,XS2376095068,2022-01-04,41.19,41.36,41.19,41.36,0,0.77
15946,XS2376095068,2022-01-05,40.99,41.02,40.77,41.02,250,-0.81
15947,XS2376095068,2022-01-06,38.15,37.85,37.85,38.15,7,-7.73
